# Association_Rule 關聯規則
- 以下使用R3.6
- 使用arules套件 install.packages("arules")

In [3]:
library("arules")

自定義一份資料集

In [10]:
df = data.frame(ID=c(1,2,3,4,5,6),
                  Onion=c(1,0,0,1,1,1),
                  Potato=c(1,1,0,1,1,1),
                  Burger=c(1,1,0,0,1,1),
                  Milk=c(0,1,1,1,0,1),
                  Beer=c(0,0,1,0,1,0))

In [11]:
df

ID,Onion,Potato,Burger,Milk,Beer
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,0,0
2,0,1,1,1,0
3,0,0,0,1,1
4,1,1,0,1,0
5,1,1,1,0,1
6,1,1,1,1,0


### 設置支持度來篩選頻繁項集
- apriori(df,parameter=list(support=0.5,minlen=2))
- 上面選擇最小支持度為50%，實務上會根據樣本量及資料特性設定

In [22]:
# R需要將項集轉為logical或factor才可以計算，以下轉為factor
df[] = lapply(df,factor)
frequent_itemsets = apriori(df[c('Onion', 'Potato', 'Burger', 'Milk', 'Beer')],parameter=list(support=0.5,minlen=2))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.8    0.1    1 none FALSE            TRUE       5     0.5      2
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 3 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[10 item(s), 6 transaction(s)] done [0.00s].
sorting and recoding items ... [5 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing ... [11 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [23]:
# R要使用inspect語法顯示結果
inspect(frequent_itemsets)

     lhs                   rhs        support   confidence lift count
[1]  {Onion=1}          => {Potato=1} 0.6666667 1.0        1.2  4    
[2]  {Potato=1}         => {Onion=1}  0.6666667 0.8        1.2  4    
[3]  {Burger=1}         => {Potato=1} 0.6666667 1.0        1.2  4    
[4]  {Potato=1}         => {Burger=1} 0.6666667 0.8        1.2  4    
[5]  {Beer=0}           => {Potato=1} 0.6666667 1.0        1.2  4    
[6]  {Potato=1}         => {Beer=0}   0.6666667 0.8        1.2  4    
[7]  {Onion=1,Burger=1} => {Potato=1} 0.5000000 1.0        1.2  3    
[8]  {Onion=1,Beer=0}   => {Potato=1} 0.5000000 1.0        1.2  3    
[9]  {Burger=1,Beer=0}  => {Potato=1} 0.5000000 1.0        1.2  3    
[10] {Milk=1,Beer=0}    => {Potato=1} 0.5000000 1.0        1.2  3    
[11] {Potato=1,Milk=1}  => {Beer=0}   0.5000000 1.0        1.5  3    


In [26]:
# 將結果排序，找出lift值最相關者
sort.rule = sort(frequent_itemsets, by=c("lift"))
inspect(sort.rule)

     lhs                   rhs        support   confidence lift count
[1]  {Potato=1,Milk=1}  => {Beer=0}   0.5000000 1.0        1.5  3    
[2]  {Potato=1}         => {Onion=1}  0.6666667 0.8        1.2  4    
[3]  {Potato=1}         => {Burger=1} 0.6666667 0.8        1.2  4    
[4]  {Potato=1}         => {Beer=0}   0.6666667 0.8        1.2  4    
[5]  {Onion=1}          => {Potato=1} 0.6666667 1.0        1.2  4    
[6]  {Burger=1}         => {Potato=1} 0.6666667 1.0        1.2  4    
[7]  {Beer=0}           => {Potato=1} 0.6666667 1.0        1.2  4    
[8]  {Onion=1,Burger=1} => {Potato=1} 0.5000000 1.0        1.2  3    
[9]  {Onion=1,Beer=0}   => {Potato=1} 0.5000000 1.0        1.2  3    
[10] {Burger=1,Beer=0}  => {Potato=1} 0.5000000 1.0        1.2  3    
[11] {Milk=1,Beer=0}    => {Potato=1} 0.5000000 1.0        1.2  3    


返回的是各個的指標的數值，可以按照感興趣的指標排序觀察,但具體解釋還得參考實際數據的含義。

In [27]:
rules.sub <- subset(frequent_itemsets, subset = lift > 1.125 & confidence > 0.8)
inspect(rules.sub)

    lhs                   rhs        support   confidence lift count
[1] {Onion=1}          => {Potato=1} 0.6666667 1          1.2  4    
[2] {Burger=1}         => {Potato=1} 0.6666667 1          1.2  4    
[3] {Beer=0}           => {Potato=1} 0.6666667 1          1.2  4    
[4] {Onion=1,Burger=1} => {Potato=1} 0.5000000 1          1.2  3    
[5] {Onion=1,Beer=0}   => {Potato=1} 0.5000000 1          1.2  3    
[6] {Burger=1,Beer=0}  => {Potato=1} 0.5000000 1          1.2  3    
[7] {Milk=1,Beer=0}    => {Potato=1} 0.5000000 1          1.2  3    
[8] {Potato=1,Milk=1}  => {Beer=0}   0.5000000 1          1.5  3    


得到幾條比較有價值的結果：
- (洋蔥和馬鈴薯)、(漢堡和馬鈴薯)可以搭配著來賣
- 如果洋蔥和漢堡都在購物籃中, 顧客買馬鈴薯的可能性也滿高的，如果他籃子裡面沒有，可以推薦一下.

### one-hot-encoding 編碼轉換
- 實務資料通常需要轉換成 one hot encoding
- 下面以逗號分隔的文字資料，為範例演練

In [50]:
retail = structure(list(ID=c(1,2,3,4,5,6),
                                        Basket=c("'Beer','Diaper','Pretzels','Chips','Aspirin'",
                                                 "'Diaper','Beer','Chips','Lotion','Juice','BabyFood','Milk'",
                                                 "'Soda','Chips','Milk'",
                                                 "'Soup','Beer','Diaper','Milk','IceCream'",
                                                 "'Soda','Coffee','Milk','Bread'",
                                                 "'Beer','Chips'")),
                                   row.names = c(NA, 6),
                                   class = "data.frame")
retail

,ID,Basket
,<dbl>,<chr>
1,1,"'Beer','Diaper','Pretzels','Chips','Aspirin'"
2,2,"'Diaper','Beer','Chips','Lotion','Juice','BabyFood','Milk'"
3,3,"'Soda','Chips','Milk'"
4,4,"'Soup','Beer','Diaper','Milk','IceCream'"
5,5,"'Soda','Coffee','Milk','Bread'"
6,6,"'Beer','Chips'"


In [51]:
# 先把ID欄位抽出來
retail_id = retail['ID']
retail_id

,ID
,<dbl>
1,1
2,2
3,3
4,4
5,5
6,6


In [52]:
# 用逗號分隔每個字詞
library(qdapTools)
retail_Basket = mtabulate(strsplit(retail$Basket, ","))
names(retail_Basket) = gsub("'", "", names(retail_Basket))
retail_Basket

Aspirin,BabyFood,Beer,Bread,Chips,Coffee,Diaper,IceCream,Juice,Lotion,Milk,Pretzels,Soda,Soup
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,1,0,1,0,1,0,0,0,0,1,0,0
0,1,1,0,1,0,1,0,1,1,1,0,0,0
0,0,0,0,1,0,0,0,0,0,1,0,1,0
0,0,1,0,0,0,1,1,0,0,1,0,0,1
0,0,0,1,0,1,0,0,0,0,1,0,1,0
0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [53]:
# 把ID欄位組回來
retail = cbind(retail_id,retail_Basket)
retail

,ID,Aspirin,BabyFood,Beer,Bread,Chips,Coffee,Diaper,IceCream,Juice,Lotion,Milk,Pretzels,Soda,Soup
,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0
2,2,0,1,1,0,1,0,1,0,1,1,1,0,0,0
3,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0
4,4,0,0,1,0,0,0,1,1,0,0,1,0,0,1
5,5,0,0,0,1,0,1,0,0,0,0,1,0,1,0
6,6,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [60]:
#一樣先轉
frequent_itemsets_2 = apriori(retail[,-1],parameter=list(support=0.5,minlen=2))
inspect(frequent_itemsets_2)

Warning message:
"Column(s) 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14 not logical or factor. Applying default discretization (see '? discretizeDF')."


ERROR: Error in discretizeDF(from): Problem with column Aspirin
Error in discretize(x = c(1L, 0L, 0L, 0L, 0L, 0L)) : 
  The calculated breaks are: 0, 0, 0, 1
  Some breaks are not unique. Change the number of breaks or consider using method 'fixed'.



     lhs                   rhs        support   confidence lift count
[1]  {Onion=1}          => {Potato=1} 0.6666667 1.0        1.2  4    
[2]  {Potato=1}         => {Onion=1}  0.6666667 0.8        1.2  4    
[3]  {Burger=1}         => {Potato=1} 0.6666667 1.0        1.2  4    
[4]  {Potato=1}         => {Burger=1} 0.6666667 0.8        1.2  4    
[5]  {Beer=0}           => {Potato=1} 0.6666667 1.0        1.2  4    
[6]  {Potato=1}         => {Beer=0}   0.6666667 0.8        1.2  4    
[7]  {Onion=1,Burger=1} => {Potato=1} 0.5000000 1.0        1.2  3    
[8]  {Onion=1,Beer=0}   => {Potato=1} 0.5000000 1.0        1.2  3    
[9]  {Burger=1,Beer=0}  => {Potato=1} 0.5000000 1.0        1.2  3    
[10] {Milk=1,Beer=0}    => {Potato=1} 0.5000000 1.0        1.2  3    
[11] {Potato=1,Milk=1}  => {Beer=0}   0.5000000 1.0        1.5  3    
